# Pretraining both network parts seperatly

In [1]:
from src.dlc_practical_prologue import generate_pair_sets
from src.utils import load_class_data, load_target_data, load_all_data
from src.models import *
from src.trainer import Trainer
import torch
import matplotlib.pyplot as plt

In [2]:
# Load data with class and larget than as targets
dl_train_all, dl_val_all, dl_test_all = load_all_data()

# Load data with class as target
dl_train_class, dl_val_class, dl_test_class = load_class_data()

In [3]:
print("--- Pre-training tail model ---\n")

tail = TailLinear(label_encoded=True)
trainer = Trainer(nb_epochs=5)
trainer.fit(tail, dl_train_all, dl_val_all)

print("\n--- Pre-train classification model ---\n")

le_net = LeNet()
trainer = Trainer(nb_epochs=15)
trainer.fit(le_net, dl_train_class, dl_val_class)

print("\n--- Putting them together ---\n")

tail.label_encoded = False
siamese = Siamese(auxiliary=le_net, target=tail, weight_aux=0.3)
# trainer = Trainer(nb_epochs=5)
# trainer.fit(siamese, dl_train_all, dl_val_all)

print("\n--- Testing model ---\n")
trainer.test(le_net, dl_test_class)
trainer.test(siamese, dl_test_all)


--- Pre-training tail model ---

# Epoch 1/5:	 loss=0.66	 loss_val=0.61	 acc_val=87.05
# Epoch 2/5:	 loss=0.54	 loss_val=0.41	 acc_val=94.2
# Epoch 3/5:	 loss=0.29	 loss_val=0.19	 acc_val=96.88
# Epoch 4/5:	 loss=0.13	 loss_val=0.08	 acc_val=100.0
# Epoch 5/5:	 loss=0.06	 loss_val=0.04	 acc_val=100.0

--- Pre-train classification model ---

# Epoch 1/15:	 loss=1.42	 loss_val=0.58	 acc_val=82.69
# Epoch 2/15:	 loss=0.44	 loss_val=0.31	 acc_val=91.83
# Epoch 3/15:	 loss=0.29	 loss_val=0.24	 acc_val=92.31
# Epoch 4/15:	 loss=0.23	 loss_val=0.26	 acc_val=91.59
# Epoch 5/15:	 loss=0.17	 loss_val=0.2	 acc_val=94.47
# Epoch 6/15:	 loss=0.15	 loss_val=0.15	 acc_val=94.95
# Epoch 7/15:	 loss=0.12	 loss_val=0.14	 acc_val=95.67
# Epoch 8/15:	 loss=0.1	 loss_val=0.15	 acc_val=96.39
# Epoch 9/15:	 loss=0.07	 loss_val=0.12	 acc_val=95.91
# Epoch 10/15:	 loss=0.06	 loss_val=0.12	 acc_val=96.15
# Epoch 11/15:	 loss=0.05	 loss_val=0.2	 acc_val=94.71
# Epoch 12/15:	 loss=0.05	 loss_val=0.14	 acc_val=96.

78.91